In [1]:
import sys
sys.path.append("..")
import numpy as np
import keras.backend as K
import nea.asap_reader as dataset 
from nea.asap_reader import read_org_dataset
import nea.utils as U
import os
from time import time
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list = "1"
set_session(tf.Session(config=config))
import nlp_feature

Using TensorFlow backend.


In [2]:
train_path = '../../data/fold_0/train.tsv'
dev_path = '../../data/fold_0/dev.tsv'
test_path = '../../data/fold_0/test.tsv'
out_dir = 'output'
vocab_size = 4000
maxlen = 0
prompt_id = int(1)

In [3]:
from keras.preprocessing import sequence
# data_x is a list of lists
(train_x, train_y, train_pmt), (dev_x, dev_y, dev_pmt), (test_x, test_y, test_pmt), vocab, vocab_size, overal_maxlen, num_outputs = dataset.get_data(
(train_path, dev_path, test_path), prompt_id, vocab_size, maxlen, tokenize_text=True, to_lower=True, sort_by_len=False, vocab_path=None)

In [4]:
from keras.preprocessing import sequence
train_x = sequence.pad_sequences(train_x, maxlen=overal_maxlen)
dev_x = sequence.pad_sequences(dev_x, maxlen=overal_maxlen)
test_x = sequence.pad_sequences(test_x, maxlen=overal_maxlen)

In [5]:
train_y = np.array(train_y, dtype=K.floatx())
dev_y = np.array(dev_y, dtype=K.floatx())
test_y = np.array(test_y, dtype=K.floatx())
train_pmt = np.array(train_pmt, dtype='int32')
dev_pmt = np.array(dev_pmt, dtype='int32')
test_pmt = np.array(test_pmt, dtype='int32')
# We need the dev and test sets in the original scale for evaluation
dev_y_org = dev_y.astype(dataset.get_ref_dtype())
test_y_org = test_y.astype(dataset.get_ref_dtype())
# Convert scores to boundary of [0 1] for training and evaluation (loss calculation)
train_y = dataset.get_model_friendly_scores(train_y, train_pmt)
dev_y = dataset.get_model_friendly_scores(dev_y, dev_pmt)
test_y = dataset.get_model_friendly_scores(test_y, test_pmt)

In [6]:
from keras.models import Sequential, Model , Input
from keras.layers import Dense, Embedding, Activation , Flatten  , concatenate , BatchNormalization
from keras.layers.merge import Concatenate
from keras.layers import Dropout, GlobalAveragePooling1D ,GlobalMaxPooling1D , MaxPooling1D , AveragePooling1D , GlobalMaxPooling1D
from keras.layers import LSTM , Bidirectional , Convolution1D
from nea.optimizers import get_optimizer

In [7]:
train_org = read_org_dataset(train_path , prompt_id ,tokenize_text=True, to_lower=True )
dev_org = read_org_dataset(dev_path , prompt_id ,tokenize_text=True, to_lower=True )
test_org = read_org_dataset(test_path , prompt_id ,tokenize_text=True, to_lower=True )

In [8]:
def get_feature(org_array):
    corpus = ' '.join(org_array)
    f1 = nlp_feature.differe_level_feature(org_array)
    f2 = nlp_feature.corpus_word_level_score(org_array)
    f3 = nlp_feature.avg_sentence_length(corpus)
    f4 = nlp_feature.punct_feature(corpus)
    f5 = nlp_feature.avg_word_length(corpus)
    f6 = nlp_feature.count_connectives(corpus)
    f7 = nlp_feature.char_count(corpus)
    f8 = nlp_feature.lexicon_count(corpus)
    f9 = nlp_feature.flesch_reading_ease(corpus)
    f10 = nlp_feature.smog_index(corpus)
    f11 = nlp_feature.automated_readability_index(corpus)
    #F = [f3 , f5 , f6 , f7 , f8]
    F = [*f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11]
    #F = [*f1 , f2 , f5 , f6 ,f7 , f8]
    #F = [*f1 , f9 , f10 , f11]
    #F = [*f1]
    return F 

In [9]:
from sklearn.preprocessing import StandardScaler , MinMaxScaler
#scaler = StandardScaler() 
scaler  = MinMaxScaler()

In [10]:
train_f = np.array([get_feature(x) for x in train_org] , dtype=K.floatx())
dev_f =  np.array([get_feature(x) for x in dev_org] , dtype=K.floatx())
test_f = np.array([get_feature(x) for x in test_org] , dtype=K.floatx())

In [11]:
scaler.fit(np.concatenate((train_f,dev_f,test_f), axis=0))
#scaler.fit(train_f)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [12]:
train_f = np.array(scaler.transform(train_f) , dtype=K.floatx())
dev_f = np.array(scaler.transform(dev_f) , dtype=K.floatx())
test_f = np.array(scaler.transform(test_f) , dtype=K.floatx())

In [18]:
main_input = Input(shape=(overal_maxlen,), dtype='float64')
embedder = Embedding(len(vocab), 50, input_length = overal_maxlen) 
embed = embedder(main_input)
cnn1 = Convolution1D(filters=50 ,kernel_size=3)(embed)
cnn1 = MaxPooling1D(pool_size=5)(cnn1)
flat = Flatten()(cnn1)
######################
#Other_Input = Dense(30 ,activation='relu', input_shape=(len(train_f[0]),))
Other_Input = Input(shape=(len(train_f[0]),))
D1 = Dense(50)(Other_Input)
#add_feature = concatenate([flat, Other_Input],axis=1)
add_feature = concatenate([flat, D1],axis=1)
##########################################
main_output = Dense(1, activation='sigmoid')(add_feature)

model = Model(inputs = [main_input , Other_Input], outputs = main_output)

In [19]:
loss = 'mean_squared_error'
metric = 'mean_absolute_error'
opt = get_optimizer('adam')

In [23]:
model.compile(loss=loss,
              optimizer=opt,
              metrics=[metric])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 939)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 939, 50)      200000      input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 937, 50)      7550        embedding_2[0][0]                
__________________________________________________________________________________________________
max_pooling1d_2 (MaxPooling1D)  (None, 187, 50)      0           conv1d_2[0][0]                   
__________________________________________________________________________________________________
input_4 (I

In [24]:
from nea.evalute_twoinput import Evaluator 
evl = Evaluator (dataset, prompt_id, out_dir, dev_x , dev_f  ,test_x , test_f , dev_y, test_y, dev_y_org, test_y_org)
evl.evaluate(model, -1, print_info=True)

[Dev]   loss: 0.0062, metric: 0.0609, mean: 8.556 (8.508), stdev: 1.287 (1.539)
[Test]  loss: 0.0062, metric: 0.0604, mean: 8.515 (8.473), stdev: 1.278 (1.538)
[DEV]   QWK:  0.820, LWK: 0.627, PRS: 0.860, SPR: 0.849, Tau: 0.716 (Best @ -1: {{0.820}}, 0.627, 0.860, 0.849, 0.716)
[TEST]  QWK:  0.823, LWK: 0.631, PRS: 0.861, SPR: 0.834, Tau: 0.698 (Best @ -1: {{0.823}}, 0.631, 0.861, 0.834, 0.698)
--------------------------------------------------------------------------------------------------------------------------


In [25]:
total_train_time = 0
total_eval_time = 0
best , count = 0 , 0
for ii in range(100):
    # Training
    t0 = time()
    train_history = model.fit([train_x , train_f], train_y, batch_size=32, epochs=1, verbose=1)
    #train_history = model.fit([train_f], train_y, batch_size=32, epochs=1, verbose=1)
    tr_time = time() - t0
    total_train_time += tr_time
    # Evaluate
    t0 = time()
    evl.evaluate(model, ii)
    evl_time = time() - t0
    total_eval_time += evl_time
    train_loss = train_history.history['loss'][0]
    train_metric = train_history.history[metric][0]
    print('Epoch %d, train: %is, evaluation: %is' % (ii, tr_time, evl_time))
    print('[Train] loss: %.4f, metric: %.4f' % (train_loss, train_metric))
    evl.print_info()
    
evl.print_final_info()

Epoch 1/1
1070/1070 [==============================] - 0s 419us/step - loss: 0.0054 - mean_absolute_error: 0.0573
Epoch 0, train: 0s, evaluation: 0s
[Train] loss: 0.0054, metric: 0.0573
[Dev]   loss: 0.0065, metric: 0.0630, mean: 8.481 (8.508), stdev: 1.454 (1.539)
[Test]  loss: 0.0063, metric: 0.0614, mean: 8.445 (8.473), stdev: 1.444 (1.538)
[DEV]   QWK:  0.839, LWK: 0.646, PRS: 0.857, SPR: 0.851, Tau: 0.720 (Best @ 0: {{0.839}}, 0.646, 0.857, 0.851, 0.720)
[TEST]  QWK:  0.841, LWK: 0.656, PRS: 0.860, SPR: 0.839, Tau: 0.703 (Best @ 0: {{0.841}}, 0.656, 0.860, 0.839, 0.703)
--------------------------------------------------------------------------------------------------------------------------
Epoch 1/1
1070/1070 [==============================] - 0s 206us/step - loss: 0.0026 - mean_absolute_error: 0.0386
Epoch 1, train: 0s, evaluation: 0s
[Train] loss: 0.0026, metric: 0.0386
[Dev]   loss: 0.0064, metric: 0.0617, mean: 8.348 (8.508), stdev: 1.168 (1.539)
[Test]  loss: 0.0066, metric: